In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import shutil
import glob
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.track as track
import thor.option as option
import thor.tag as tag
import thor.visualize as visualize
import thor.synth as synth

notebook_name = "cpol_demo.ipynb"


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [3]:
# Parent directory for saving outputs
base_local = Path.home() / "THOR_output"

lat = np.arange(0, 5+0.025, 0.025).tolist()
lon = np.arange(0, 5+0.025, 0.025).tolist()
grid_options = grid.create_options(name="geographic", latitude=lat, longitude=lon)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, filename="synth_geographic")

2024-08-27 17:56:06,088 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/options/grid_options/synth_geographic.yaml


In [7]:
ds = synth.synth.dataset(grid_options)

In [ ]:

# Create the tag_options dictionary
era5_pl_tag_options = data.era5.tag_options()
era5_sl_tag_options = data.era5.tag_options(dataset="era5_sl")
tag_options = option.consolidate_options([era5_pl_tag_options, era5_sl_tag_options])
tag.save_tag_options(tag_options, filename="era5")

# Create the track_options dictionary
track_options = option.mcs(dataset="cpol", tags=["era5_pl", "era5_sl"])
option.save_track_options(track_options, filename="cpol_mcs")

# Create the display_options dictionary
visualize_options = {
    obj: visualize.option.runtime_options(obj, save=True, style="presentation")
    for obj in ["cell", "anvil", "mcs"]
}
visualize_options["middle_cloud"] = visualize.option.runtime_options(
    "middle_cloud", save=True, style="presentation", figure_types=["mask"]
)
visualize.option.save_display_options(visualize_options, filename="runtime_mcs")
visualize_options = None

In [5]:
tag_options = None

output_directory = base_local / "runs/cpol_demo_geographic"
if output_directory.exists():
    shutil.rmtree(output_directory)
times = data.utils.generate_times(data_options["cpol"])
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    tag_options,
    visualize_options,
    output_directory=output_directory
)

2024-08-27 16:34:30,474 - thor.track - DEBUG - Beginning simultaneous tracking.
2024-08-27 16:34:30,580 - thor.track - INFO - Processing 2005-11-13T14:00:09.
2024-08-27 16:34:30,581 - thor.data.aura - DEBUG - Updating cpol dataset for 2005-11-13T14:00:09.
2024-08-27 16:34:30,582 - thor.data.aura - DEBUG - Converting cpol data from twp10cpolgrid150.b2.20051113.140000.nc
2024-08-27 16:34:30,613 - thor.data.aura - DEBUG - Creating new geographic grid with spacing 0.025 m, 0.025 m.
2024-08-27 16:34:33,215 - thor.track - DEBUG - Processing hierarchy level 0.
2024-08-27 16:34:33,216 - thor.track - DEBUG - Tracking cell.
2024-08-27 16:34:42,249 - thor.match.match - DEBUG - Matching cell objects.
2024-08-27 16:34:42,250 - thor.match.match - DEBUG - No previous mask, or no objects in previous mask.
2024-08-27 16:34:42,255 - thor.visualize.runtime - DEBUG - Generating runtime visualizations.
2024-08-27 16:34:42,256 - thor.track - DEBUG - Tracking middle_cloud.
2024-08-27 16:34:42,260 - thor.visu